## 무인택시 배차 
---
K모빌리티는 수년간의 연구끝에 무인자율주행택시를 개발했다. 그리고 테스트목적으로 자율주행택시 **1대**를 서울,경기 지역에서 운행 하려고한다. 그런데 아직 시범운행이라서 택시는 예약제로 운행이된다. 즉, 택시Call정보를 테스트시작 전 이미 가지고 있다. K모빌리티 데이터과학자 윤우의 임무는 자율주행택시가 요금을 최대한 많이 받아오도록 만드는 것이다. 윤우를 도와 자율주행택시의 **수익 최대화** 알고리즘을 만들어보자.
___
### 입력
입력의 첫째 줄에 모든 택시콜 **위치개수**($N$)와 &nbsp; **마지막택시콜 출발시간**($T$)이 주어진다.<br>
입력의 둘째 줄부터 **$(N \times N - N)\times T$** 줄에 걸쳐 각 줄마다 아래와 같은 정보가 주어진다.<br>
각 줄의 정보는 택시콜정보로써, **출발위치($P_{start}$), 목적지위치($P_{arrive}$), 출발시간($T_{start}$), 주행시간($T_{take}$), 요금($F$)** 가주어진다.<br>
- 위치정보 $P$는 정수형이며, $1\leq P_i\leq100$ 이다<br>
- 시간정보 $T$는 정수형이며, $1\leq T_i\leq100$ 이다<br>
- 요금정보 $F$는 정수형이며, $0\leq F_i\leq100$ 이다<br>
- $(P_{start},P_{arrive},T_{start})$의 모든 가능한 조합에 대한 $(T_{take},F)$이 주어질 것이다. ($P_{start},P_{arrive}$ 같은경우 제외)
- 출발위치와 목적지위치가 같은 경우 $(T_{take},F) = (0,0)$ 이라고 가정한다. (입력에서는 주어지지 않음)
- 요금과 주행시간이 비례하지만, 약간의 Noise가 있다.
- 택시의 첫 운행은 어느곳에서나 시작 가능하다.
- 택시는 현재위치에서 대기할 수 있으며, T는 1 흐른다.
___
### 출력
운행을통해 얻을수있는 요금합의 최댓값과, 운행루트를 출력 2줄에 걸쳐 출력한다.(운행루트는 공백을 사이로 출력한다)

---
### 예제
#### 입력
    2 10
    1 2 1 9 9
    1 2 2 5 1
    1 2 3 4 7
    1 2 4 3 6
    1 2 5 2 5
    1 2 6 8 5
    1 2 7 6 8
    1 2 8 3 4
    1 2 9 2 1
    1 2 10 3 1
    2 1 1 1 2
    2 1 2 6 1
    2 1 3 4 4
    2 1 4 4 7
    2 1 5 1 1
    2 1 6 6 2
    2 1 7 5 6
    2 1 8 3 1
    2 1 9 4 15
    2 1 10 1 5
#### 출력
    24
    2 1 1 2 2 2 1

In [7]:
import random
import numpy as np
import sys
sys.setrecursionlimit(100*100*10000)
def EnvGenerator(N,T):
    k = 0
    Ts = np.random.normal(10, 3, T*N*N)
    for P_start in range(1,N+1):
        for P_arrive in range(1,N+1):
            for T_i in range(1,T+1):
                k += 1
                if P_start == P_arrive:
                    continue
                else:
                    T_take = min(max(int(Ts[k-1]),1),100)
                    fair = min(max(int(np.random.normal(T_take,5,1)),1),100)
                    
                    yield (P_start,P_arrive,T_i,T_take,fair)

### 문제 풀이
$V(T,P)$ = T시에 P위치에서 얻을 수 있는 최대 수익 <br>
$TF(T,P_{s},P_{a}) = $ $T$시에 $P_s$에서 출발하여 $P_a$에 도착할때 걸리는 \[시간,요금\], 2차원 벡터 
$  \begin{bmatrix}
    t  \\
    f 
  \end{bmatrix} \in \mathbb{N}^{2}$
<br>

---
#### 점화식 
$$V(T,P_s) = \max_{P_a}\Big( [0,1 ] \cdot TF(T,P_{s},P_{a}) + V\big(T+[1,0] \cdot TF(T,P_{s},P_{a}), P_a \big)\Big)$$

In [8]:
N = 100; T = 100
y = EnvGenerator(N,T)
"""
TF <- (T_take, fair) when Start from Ps to Pa in time T.
"""
TF = [[[(0,0) for _ in range(N+1)] for _ in range(N+1)] for _ in range(T+1)]

"""
V_tp <- optimal expected sum of fairs when the taxi initiates in time t at position p
"""
V_tp = [[-1 for _ in range(N+1)] for _ in range(T+1)]
for _ in range((N*N-N)*T):
    P_start,P_arrive,T_start,T_take,fair =next(y)
    #print(P_start,P_arrive,T_start,T_take,fair)
    TF[T_start][P_start][P_arrive] = (T_take,fair)
route_T_Ps = [[-1 for _ in range(N+1)] for _ in range(T+1)]
first_val = -1
first_loc = -1
def V(t,ps):
    global V_tp,TF,first_val,first_loc
    if t > T:
        return 0
    if V_tp[t][ps] != -1:
        return V_tp[t][ps]
    max_val = V(t+1,ps)
    max_pa = ps
    for pa,tfs in enumerate(TF[t][ps]):
        if pa == 0 or pa==ps:
            continue
        t_take,fair = tfs
        val = fair + V(t+t_take,pa)
        if val >= max_val:
            max_val = val
            max_pa = pa
    V_tp[t][ps] = max_val
    route_T_Ps[t][ps] = max_pa
    if max_val > first_val:
        first_val = max_val
        first_loc = ps
    return V_tp[t][ps] 
    

In [9]:
ans = max([V(1,i) for i in range(1,N+1)])

In [10]:
i = 1
curloc = first_loc
val_sum = 0
visits = []
earned = [0]
while True:
    visits.append(str(curloc))
    nextloc=route_T_Ps[i][curloc]
    fair_i=TF[i][curloc][nextloc][1]
    earned.append(earned[-1]+fair_i)
    val_sum += fair_i
    i += max(TF[i][curloc][nextloc][0],1)
    if val_sum == ans:
        visits.append(str(nextloc))
        break
    curloc=nextloc

In [11]:
print(ans)
print(" ".join(visits))

438
45 10 4 32 29 29 61 42 43 30 79 30 51 51 46 93 84 67 67 97 97 41 75 52 41 41 41 48 92 98 38 92 67 92 100 36 89 92 92 41 41 7 1 11 11 8 5 5 68 16 26 26 62
